# Описание проекта

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.
Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.


### Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
*   `сalls` — количество звонков,
*   `minutes` — суммарная длительность звонков в минутах,
*   `messages` — количество sms-сообщений,
*   `mb_used` — израсходованный интернет-трафик в Мб,
*   `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).


## Постановка задачи
### [Шаг 1. Знакомство с данными и изучение общей информации](#section1)
### [Шаг 2. Подготовка данных](#section2)
Разделите исходные данные на обучающую, валидационную и тестовую выборки.

### [Шаг 3. Исследование разных моделей машинного обучения](#section3)
Исследуйте качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.

### [Шаг 4. Оценка качества моделей](#section4)
Проверьте качество модели на тестовой выборке.

### [Шаг 5. Дополнительное задание](#section5)
Дополнительное задание: проверьте модели на вменяемость. Ничего страшного, если не получится: эти данные сложнее тех, с которыми вы работали раньше. В следующем курсе подробнее об этом расскажем.

### [Шаг 6. Общий вывод](#section6)

# Решение задачи
## Шаг 1. Знакомство с данными и изучение общей информации. <a class="anchor" id="section1"></a>

Импортируем необходимые библиотеки для дальнейшей работы.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
import warnings

warnings.filterwarnings('ignore')

Чтение файла с данными и вывод первых пяти строчек таблицы для ознакомления.

In [2]:
# users_data = pd.read_csv('/datasets/users_behavior.csv')
users_data = pd.read_csv('users_behavior.csv')
users_data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Получение общей информации о датафрейме.

In [3]:
users_data.info()

Все столбцы заполнены, явных пропусков не наблюдается, предобработка данных в данном случае не требуется. Посмотрим на пять случайных строк для знакомства с содержимым данных.

In [4]:
users_data.sample(n=5)

,calls,minutes,messages,mb_used,is_ultra
2144,44.0,324.86,0.0,18611.43,0
902,98.0,657.01,70.0,13309.90,0
1223,93.0,671.14,145.0,21441.95,1
1681,12.0,76.97,0.0,1893.18,0
3213,80.0,566.09,6.0,29480.52,1


**Наблюдение:**
Перед нами стоит задача классификации. За целевой признак берем столбец is_ultra. Остальные признаки помогут нам предсказывать решение по столбцу is_ultra.

## Шаг 2. Подготовка данных <a class="anchor" id="section2"></a>
Разделите исходные данные на обучающую, валидационную и тестовую выборки.
Разобьем выборку по принципу 60/20/20.

In [5]:
features_train_data, features_test_data, target_train_data, target_test_data = train_test_split(
    users_data.drop(['is_ultra'], axis=1), users_data['is_ultra'], test_size=0.2, random_state=42)

features_train_data, features_valid_data, target_train_data, target_valid_data = train_test_split(
    features_train_data, target_train_data, test_size=0.25, random_state=42)

In [6]:
print('Размер обучающей выборки', features_train_data.shape[0])
print('Размер валидационной выборки', features_valid_data.shape[0])
print('Размер тестовой выборки', features_test_data.shape[0])

**Итог:**
Данные были разбиты по принципу 60/20/20.
- Обучение пройдет на данных `features_train_data` и `target_train_data`;
- Валидация модели на данных `features_valid_data` и `target_valid_data`;
- Лучшая модель по валидации будет применена на данных `features_test_data` и `target_test_data`.

## Шаг 3. Исследование разных моделей машинного обучения <a class="anchor" id="section3"></a>
Исследуйте качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.

Изменяя гиперпараметры найдем наилучшую модель решающего дерева.

In [7]:
dtc_depth, dtc_accuracy = 0, 0
dtc_criterion, dtc_splitter = None, None
dtc_model = None

for depth in range(1, 15):
    for criterion in ['gini', 'entropy']:
        for splitter in ['best', 'random']:
            model = DecisionTreeClassifier(random_state=42, max_depth=depth, criterion=criterion, splitter=splitter)
            model.fit(features_train_data, target_train_data)
            predictions_valid_data = model.predict(features_valid_data)
            accuracy = accuracy_score(target_valid_data, predictions_valid_data)
            if accuracy > dtc_accuracy:
                dtc_depth = depth
                dtc_accuracy = accuracy
                dtc_criterion = criterion
                dtc_splitter = splitter
                dtc_model = model

print('Лучшая глубина дерева = {}, лучшая точность = {}, критерий: {}, стратегия разделения: {}'
      .format(dtc_depth, dtc_accuracy, dtc_criterion, dtc_splitter))

Перебрав возможные параметры гиперпараметров нашли модель дерева решений с лучшими параметрами.

***Примечание:*** не получилось проверить параметр `log_loss`, получила ошибку `KeyError`.

Изменяя гиперпараметры найдем наилучшую модель случайного леса.

In [8]:
rfc_est, rfc_depth, rfc_accuracy = 0, 0, 0
rfc_criterion = None
rfc_model = None

for depth in range(1, 20):
    for est in range(1, 20):
        for criterion in ['gini', 'entropy']:
            model = RandomForestClassifier(random_state=42, max_depth=depth, n_estimators=est, criterion=criterion)
            model.fit(features_train_data, target_train_data)
            predictions_valid_data = model.predict(features_valid_data)
            # result = model.score(features_valid_data, target_valid_data)
            accuracy = accuracy_score(target_valid_data, predictions_valid_data)
            if accuracy > rfc_accuracy:
                rfc_depth = depth
                rfc_est = est
                rfc_accuracy = accuracy
                rfc_criterion = criterion
                rfc_model = model

print('Лучшее количество деревьев = {}, глубина дерева = {}, лучшая точность = {}, критерий: {}'
      .format(rfc_est, rfc_depth, rfc_accuracy, rfc_criterion))

Изменяя гиперпараметры найдем наилучшую модель линейной регрессии.

In [15]:
lr_max_iter, lr_accuracy, lr_float_c = 0, 0, 0
lr_solver = None
lr_model = None
for float_c in [x / 10 for x in range(1, 21)]:
    for solver in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:
        model = LogisticRegression(random_state=42, C=float_c, solver=solver)
        model.fit(features_train_data, target_train_data)
        model.fit(features_train_data, target_train_data)
        predictions_valid_data = model.predict(features_valid_data)
        accuracy = accuracy_score(target_valid_data, predictions_valid_data)
        if accuracy > lr_accuracy:
            lr_accuracy = accuracy
            lr_model = model
            lr_float_c = float_c
            lr_solver = solver

print('Лучшая регуляризация = {}, лучшая точность = {}, алгоритм: {}'
      .format(lr_float_c, lr_accuracy, lr_solver))

**Итог:**
В ходе исследования были получены следующие данные:

Точность модели решающего дерева при глубине 8 равна 0.811.
Точность модели случайного леса при количестве деревьев 4 и глубине 7 равна 0.814.
Точность модели логистической регрессии равна 0.745.
Так как выборка небольшая, то лучший результат показывает случайный лес с гиперпараметрами:
- количество деревьев = 4,
- глубина дерева = 7,
- критерий: entropy.

## Шаг 4. Оценка качества моделей <a class="anchor" id="section4"></a>
Проверьте качество модели на тестовой выборке.

In [10]:
dtc_test_predictions = dtc_model.predict(features_test_data)
dtc_test_accuracy = accuracy_score(target_test_data, dtc_test_predictions)
print('Точность модели решающего леса на тестовой выборке {}, разница с валидационной выборкой {}'
      .format(dtc_accuracy, abs(dtc_accuracy - dtc_test_accuracy)))

rfc_test_predictions = rfc_model.predict(features_test_data)
rfc_test_accuracy = accuracy_score(target_test_data, rfc_test_predictions)
print('Точность модели случайного леса на тестовой выборке {}, разница с валидационной выборкой {}'
      .format(rfc_accuracy, abs(rfc_accuracy - rfc_test_accuracy)))

lr_test_predictions = lr_model.predict(features_test_data)
lr_test_accuracy = accuracy_score(target_test_data, lr_test_predictions)
print('Точность модели логистической регресиии на тестовой выборке {}, разница с валидационной выборкой {}'
      .format(lr_accuracy, abs(lr_accuracy - lr_test_accuracy)))

**Итог:**
Для решения поставленной задачи можно выбрать модель случайного леса, как модель с самой высокой точностью. Учитывая разницу между текущей точностью и точностью на валидационной выборке 0.013, можно сделать вывод слабой переобученности модели.

## Шаг 5. Дополнительное задание <a class="anchor" id="section5"></a>
Дополнительное задание: проверьте модели на вменяемость.
Проведем сравнение между константным предсказанием и лучшей обученной моделью - случайным лесом.

<img src="https://upload.wikimedia.org/wikipedia/commons/b/ba/Warning_sign_4.0.png" align=left width=44, heigth=33>
<div class="alert alert-warning">
В основе проверки на адекватность лежит очень простая идея. Модель считается адекватной, если способна "побить" любой "наивный" алгоритм, например предсказывающий случайный или постояный ответ. 

Для нашего проекта самую высокую точность дает  "наивный" алгоритм, всегда предсказывающий самый популярный ответ (у нас это 0). Проверить точность такого подхода можно вручную или используя DummyClassifier.
    
Ты использовала DummyClassifier, но достаточно было просто подсчитать точность (около 0,7) и сравнить с точностями полученными с помощью наших обученных моделей, чтобы убедиться в их адекватности.
       
    
</div>

<div class="alert alert-info" role="alert">

Много лишних действий получилось и ненужной информации, да? (⁄ ⁄•⁄ω⁄•⁄ ⁄)
</div>

<img src="http://s3.amazonaws.com/pix.iemoji.com/images/emoji/apple/ios-12/256/waving-hand.png" align=left width=44, heigth=44>
<div class="alert alert-info">
<b> Комментарий ревьюера v2</b>

Да, очень сложно понять смысл твоих действий. Поэтому я и призываю к простоте. Наверное можно применить и другие метрики (после того, как ты проверишь по аккураси), но нужно больше пояснений....</div>

In [11]:
dc_model = DummyClassifier(strategy="most_frequent", random_state=42)
dc_model.fit(features_train_data, target_train_data)

target_data_true = target_test_data
dc_predictions = dc_model.predict(features_test_data)
dc_accuracy = accuracy_score(target_test_data, dc_predictions)
dc_precision = precision_score(target_test_data, dc_predictions)
dc_recall = recall_score(target_test_data, dc_predictions)
dc_f_score = f1_score(target_test_data, dc_predictions)
print('Точность(accuracy) константного предсказания на тестовой выборке {}, точность(precision) {}, '
      'полнота(recall) {}, F1-мера {}'.format(dc_accuracy, dc_precision, dc_recall, dc_f_score))
dc_mf_tn, dc_mf_fp, dc_mf_fn, dc_mf_tp = confusion_matrix(target_data_true, dc_predictions,
                                                          labels=[0, 1]).ravel()
dc_confusion_matrix = np.array([['', 'Прогнозируемый класс +', 'Прогнозируемый класс -'],
                                ['Истинный класс +', dc_mf_tp, dc_mf_fn],
                                ['Истинный класс -', dc_mf_fp, dc_mf_tn]])
dc_confusion_matrix

array([['', 'Прогнозируемый класс +', 'Прогнозируемый класс -'],
       ['Истинный класс +', '0', '188'],
       ['Истинный класс -', '0', '455']], dtype='<U22')

In [12]:
rfc_precision = precision_score(target_test_data, rfc_test_predictions)
rfc_recall = recall_score(target_test_data, rfc_test_predictions)
rfc_f_score = f1_score(target_test_data, rfc_test_predictions)
print('Точность(accuracy) модели случайного леса на тестовой выборке {}, точность(precision) {}, '
      'полнота(recall) {}, F1-мера {}'.format(rfc_accuracy, rfc_precision, rfc_recall, rfc_f_score))
rfc_mf_tn, rfc_mf_fp, rfc_mf_fn, rfc_mf_tp = confusion_matrix(target_data_true, rfc_test_predictions,
                                                              labels=[0, 1]).ravel()
rfc_confusion_matrix = np.array([['', 'Прогнозируемый класс +', 'Прогнозируемый класс -'],
                                 ['Истинный класс +', rfc_mf_tp, rfc_mf_fn],
                                 ['Истинный класс -', rfc_mf_fp, rfc_mf_tn]])
rfc_confusion_matrix

array([['', 'Прогнозируемый класс +', 'Прогнозируемый класс -'],
       ['Истинный класс +', '91', '97'],
       ['Истинный класс -', '31', '424']], dtype='<U22')

**Итог:**
Адекватность модели оценивается по нескольким параметрам. Для модели классификации это accuracy, precision и recall. Качество модели лучше всего отражают precision и recall и эти метрики складываются в F1-меру, как единую оценку модели.

Выбранная за рабочую, модель случайного леса лучше справляется, чем константное предсказание, однако гармоническое среднее между точностью и полнотой (F1-мера) получилось всего ~58%, что говорит о среднем качестве модели.

## Шаг 6. Общий вывод <a class="anchor" id="section6"></a>

В ходе проделанной работы было выполнено:
- Изучен файл с данными.
- Разбиты данные на три выборки: обучающая, валидационная и тестовая.
- Исследованы три модели классификации: Решающее дерево, Случайный лес и Логистическая регрессия.
- Найдены оптимальные параметры для каждой модели и выбрана одна из них для обучения модели.
- Оценена точность обученной модели.
- Оценена адекватность модели.

Для обучения модели было использована выборка в 2000 значений. Этого может быть недостаточно для повышения ключевых параметров модели классификатора. Увеличение в 10 или 100 увеличит качественные характеристики модели.

<img src="http://s3.amazonaws.com/pix.iemoji.com/images/emoji/apple/ios-12/256/waving-hand.png" align=left width=44, heigth=44>
<div class="alert alert-info">
<b> Заключительный комментарий</b>

В целом мне все понравилось: твоя работа выполнена на хорошем уровне с минимальными помарками.
Также работа выглядит аккуратной и хорошо оформленной.
Вижу, что тебе дается python, и инструментарий машинного обучения.
 
Едиственный момент, который нужно обязательно исправить: прошу тебя убрать из проекта использование mse. Это метрика применяется для задач регрессии и использовать её в этом проекта это серьезная ошибка.
    
Желтые комментарии на твое усмотрение.
    
Жду твой проект на финальное ревью :)
</div>

<div class="alert alert-info" role="alert">
    <b>Спасибо большое что уделил время ( ´ ∀ `)ノ～ ♡</b>

Постаралась учесть твои замечания. Было очень приятно с тобой познакомиться) И огромная благодарность за полезные ссылки, все открыла и начала читать)
</div>

<img src="http://s3.amazonaws.com/pix.iemoji.com/images/emoji/apple/ios-12/256/waving-hand.png" align=left width=44, heigth=44>
<div class="alert alert-info">
<b> Заключительный комментарий v2</b>

И снова привет! На этот раз все ОК и проект может быть принят. Поздравляю и желаю дальнейших успехов!</div>